In [1]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

import random
import shutil
import torch
import cv2 as cv
import numpy as np
import tensorflow as tf
import torchvision as tv
from PIL import Image, ImageFilter

from alzheption.clahe import Clahe
from alzheption.extractor import AlzheptionExtractor

2025-03-18 06:39:41.165033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742254781.184089  283704 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742254781.190225  283704 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-18 06:39:41.209391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def add_gaussian_noise(image, mean=0, std=25):
    np_img = np.array(image, dtype=np.float32)
    noise = np.random.normal(mean, std, np_img.shape).astype(np.float32)
    np_img = np.clip(np_img + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(np_img)

In [3]:
path = '../../asset/dataset_jpg_brightness_balance/'

dataset = tv.datasets.ImageFolder(path)

In [4]:
list_aug = ["rotate"]
dir_path = f'../../asset/dataset_jpg_brightness_balance_augmentation_{"_".join([aug for aug in list_aug])}/'
shutil.rmtree(dir_path, ignore_errors=True)
os.makedirs(dir_path, exist_ok=True)

idx_transform = 0
for idx, (img, label) in enumerate(dataset):
    name_class = dataset.classes[label]
    dir_label = os.path.join(dir_path, name_class)
    os.makedirs(dir_label, exist_ok=True)
    
    img.save(os.path.join(dir_label, f"img_{idx}_original.jpg"))
    
    transformations = {
        # "hflip": tv.transforms.functional.hflip(img),
        # "vflip": tv.transforms.functional.vflip(img),
        # "rotate_90": tv.transforms.functional.rotate(img, 90, expand=True),
        "random_rotate": tv.transforms.functional.rotate(img, random.uniform(-90, 90), expand=True),
        # "random_crop": tv.transforms.RandomResizedCrop(size=img.size, scale=(0.8, 1.0))(img),
        # "random_shear": tv.transforms.RandomAffine(degrees=0, shear=20)(img),
        # "blur": img.filter(ImageFilter.GaussianBlur(radius=random.uniform(1, 2))),
        # "brightness_contrast": tv.transforms.ColorJitter(brightness=0.2, contrast=0.2)(img),
        # # "random_noise": add_gaussian_noise(img),
        # "cutout": tv.transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), value=0)(tv.transforms.ToTensor()(img)).permute(1, 2, 0),
    }
    for aug_name, aug_img in transformations.items():
        if list(transformations.keys())[idx_transform] != aug_name:
            continue
        
        if isinstance(aug_img, torch.Tensor):  # Untuk cutout, ubah ke PIL Image
            aug_img = tv.transforms.ToPILImage()(aug_img.numpy())
        aug_img.save(os.path.join(dir_label, f"img_{idx}_{aug_name}.jpg"), format="JPEG")
    
    if idx_transform == len(transformations) - 1:
        idx_transform = 0
    else:
        idx_transform += 1

In [5]:
# img1 = tv.transforms.functional.hflip(img)
# img1.save(os.path.join(dir_label, f"img_{idx}_1.jpg"))

# img2 = tv.transforms.functional.rotate(img, 45)
# img2.save(os.path.join(dir_label, f"img_{idx}_2.jpg"))

# img3 = tv.transforms.functional.vflip(img)
# img3.save(os.path.join(dir_label, f"img_{idx}_3.jpg"))

# img4 = tv.transforms.functional.rotate(img, 90)
# img4.save(os.path.join(dir_label, f"img_{idx}_4.jpg"))

# img5 = tv.transforms.functional.rotate(img, 45)
# img5.save(os.path.join(dir_label, f"img_{idx}_5.jpg"))